In [20]:
!pip install scrapy-fake-useragent


### playnommore
http://playnomore.co.kr

In [1]:
import scrapy
import requests

from scrapy.http import TextResponse
from fake_useragent import UserAgent

In [3]:
!rm -rf playnomore
!scrapy startproject playnomore

New Scrapy project 'playnomore', using template directory '/Users/jhkim/opt/miniconda3/lib/python3.7/site-packages/scrapy/templates/project', created in:
    /Users/jhkim/Documents/dev/TIL_/playnomore

You can start your first spider with:
    cd playnomore
    scrapy genspider example example.com


In [5]:
%%writefile playnomore/playnomore/items.py
import scrapy


class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()


Overwriting playnomore/playnomore/items.py


In [8]:
url = "http://www.playnomore.co.kr/category/bag/24/"
#headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers = headers)
response = TextResponse(req.url, body = req.text, encoding = "utf-8")
response

<200 http://www.playnomore.co.kr/category/bag/24/>

In [ ]:
#links = response.xpath('//*[@id="contents"]/div[2]/ul/li/div[1]/a/@href').extreat()
#links = map(lambda data: reponse.urljoin(data), links)
#

In [27]:
links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
links = list(map(lambda data:response.urljoin(data), links))

# detail : title, price, image url

In [28]:
url = links[0]
headers = {"User-Agent": UserAgent().chrome}
req = requests.get(url, headers = headers)
response = TextResponse(req.url, body = req.text, encoding = "utf-8")
response

<200 http://www.playnomore.co.kr/product/pre-order20off-micro-baguette-yellow-180/573/?cate_no=24&display_group=1>

In [46]:
#title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
#title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
#title2 = response.xpath('')//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]
title = "".join(title1) + "".join(title2)
price = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/stirke/text()').extract()
img = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src')[0].extract()
title, price, img

('[Pre-Order/20%off] MICRO BAGUETTE yellow ',
 ['$ 180'],
 'http://www.playnomore.co.kr/web/product/big/20200407/626de3d4081df6e1dff057623e9e4877.jpg')

In [47]:
!pip install scrapy-fake-useragent

In [48]:
!pip list | grep fake

fake-useragent            0.1.11
scrapy-fake-useragent     1.2.0


# spider 

In [49]:
%%writefile playnomore/playnomore/spiders/spiders.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        "DOWNLOADER_MIDDLEWARES": {
            "scrapy.downloadmiddlewares.useragent.UserAgentMiddleware": None,
            "scrapy_fake_useragent.middleware.RandomUserAgentMiddleware": 400,
        }
    }
    
    def __init__(self,category1, category2, **kwargs): #스크래피 실행시 아규먼트 설정
        self.start_urls = ["http://palynomore.co.kr/category/{}/{}".format(category1, category2)]#스크래피 실행시 아규먼트 설정
        super.__init__(**kwargs)#스크래피 실행시 아규먼트 설정
    
    def start_requests(self):
        self.start_urls = ["http://playnomore.co.kr/category/bag/24/"]#스크래피 실행시 아규먼트 설정
        for url in urls:
            yield scrapy.Request(url, callback=self.parse)
    
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div[1]/a/@href').extract()
        links = list(map(lambda data:response.urljoin(data), links))
        for link in links:
            yield scrapy.Request(link, callback=self.parse_content)
            
    def parse_content(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()')[0].extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = title1 + " ".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()')[0].extract()
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[2]/ul/li[1]/img/@src')[0].extract()
        item["link"] = response.url
        yield item

Overwriting playnomore/playnomore/spiders/spiders.py


# arg setting

# run scrpay 

In [50]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore_new.py

Overwriting run.sh


In [51]:
!soruce run.sh

/bin/sh: soruce: command not found


In [ ]:
pd.read_csv("playnomore/playnomore.sh")